In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation
%matplotlib ipympl
import numpy as np
import pandas as pd

from scipy.spatial import KDTree, cKDTree
from scipy.ndimage import uniform_filter1d
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import time
import joblib
from tqdm import tqdm
import trackpy as tp
from numba import njit, prange

from yupi import Trajectory
import yupi.graphics as yg
import yupi.stats as ys

from utility import get_imsd, get_imsd_windowed, get_emsd, get_emsd_windowed, fit_hist, MB_2D,\
                    normal_distr, lorentzian_distr, get_trajs, speed_windowed, theta_windowed, \
                    get_smooth_trajs, get_velocities

In [2]:
traj_verb = "hough"

if traj_verb == "trackpy": 
    rawTrajs = pd.read_parquet("../tracking/results/tracking_data/trackpy_pre_merge.parquet")
    nDrops = int(len(rawTrajs.loc[rawTrajs.frame==0]))
    # Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
    smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)

    res_path = "results"
    analysis_data_path = "analysis_data"

elif traj_verb == "hough":
    rawTrajs = pd.read_parquet("../tracking/results/tracking_data/tracking_hough_trackpy_linking.parquet")
    nDrops = int(len(rawTrajs.loc[rawTrajs.frame==0]))
    # Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
    smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)

    res_path = "hough_results"
    analysis_data_path = "hough_analysis_data"
else:
    raise ValueError("traj_verb must be either 'trackpy' or 'hough'")

red_particle_idx = 17
colors = rawTrajs.loc[rawTrajs.frame == 0, 'color'].values
nFrames = int(max(rawTrajs.frame) + 1)
fps = 10
print(f"Number of Droplets: {nDrops}")
print(f"Number of Frames: {nFrames} at {fps} fps --> {nFrames/10:.2f} s")

# WINDOWED ANALYSIS PARAMETERS
window = 3200 # 320 s
stride = 100 # 10 s
print("Windowed analysis args:")

startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window
nSteps = len(startFrames)
print(f"window of {window/10} s, stride of {stride/10} s --> {nSteps} steps")

# step 10 with a 10 fps video --> 1 s
units = "px/s"
default_kwargs_blue = {"color": "#00FFFF", "ec": (0, 0, 0, 0.6), "density": True}
default_kwargs_red = {"color": "#EE4B2B", "ec": (0, 0, 0, 0.6), "density": True}

Number of Droplets: 50
Number of Frames: 32000 at 10 fps --> 3200.00 s
Windowed analysis args:
window of 320.0 s, stride of 10.0 s --> 288 steps


# MSD

In [4]:
show_verb = False
run_windowed_analysis = True
plot_verb = True
animated_plot_verb = True
save_verb = True

print(f"MSD Analysis: trajs_verb = {traj_verb}, show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/msd.ipynb
#%run ./analysis_modules/msd.py

MSD Analysis: trajs_verb = hough, show_verb = False, run_windowed_analysis = True, animated_plot_verb = True
Global IMSD
Global EMSD
Raw trajs - Blue Particles: 1.183 ± 0.002, Red Particle: 0.762 ± 0.002
Smooth trajs - Blue Particles: 1.184 ± 0.002, Red Particle: 0.765 ± 0.002
Windowed IMSD


100%|██████████| 288/288 [06:08<00:00,  1.28s/it]
17:22:31 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
17:22:31 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/mean_squared_displacement/windowed_analysis/IMSD_wind_raw.mp4
17:23:24 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
17:23:24 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/mean_squared_displacement/windowed_analysis/IMSD_wind_smooth.mp4


Windowed EMSD


17:24:19 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
17:24:19 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/mean_squared_displacement/windowed_analysis/EMSD_wind_raw.mp4
17:24:51 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
17:24:51 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/mean_squared_displacement/windowed_analysis/EMSD_wind_smooth.mp4


# VELOCITY DISTRIBUTION

$$ \huge p(v) = \frac{v}{\sigma^2} e^{-\frac{v^2}{2\sigma^2}} $$

In [8]:
show_verb = False
plot_verb = True
animated_plot_verb = True
save_verb = True

v_step = 10

print(f"Speed Analysis: trajs_verb = {traj_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_distr.ipynb

Speed Analysis: trajs_verb = hough, show_verb = False, animated_plot_verb = True

 Global speed distribution analysis
Raw Trajectories
Blue Particles σ: 2.8142698983828414 ± 0.07242741090955332
Red Particle σ: 7.622198654479617 ± 0.21007989747959196
Smooth Trajectories
Blue Particles σ: 2.656802399698486 ± 0.018146336090378336
Red Particle σ: 7.655344604966516 ± 0.14809620224532655

 Windowed speed distribution Analysis


100%|██████████| 288/288 [01:01<00:00,  4.67it/s]
18:35:48 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
18:35:48 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/speed_distribution/speed_raw_wind.mp4
18:38:04 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
18:38:04 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./hough_results/speed_distribution/speed_smooth_wind.mp4


# TURNING ANGLES DISTRIBUTIONS

In [ ]:
show_verb = False
run_windowed_analysis = True
plot_verb = True
animated_plot_verb = True
save_verb = True

# choose the distribution to fit
distribution_str = "lorentzian"

if distribution_str == "gaussian":
    distribution = normal_distr
elif distribution_str == "lorentzian":
    distribution = lorentzian_distr
    distribution_str = "lorentzian"
else:
    raise ValueError("distribution_str must be either 'gaussian' or 'lorentzian'")

print(f"Turning Angles Analysis: trajs_verb = {traj_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/turning_angles.ipynb

# VELOCITY AUTOCORRELATION

In [ ]:
show_verb = False
run_windowed_analysis = False
plot_verb = True
animated_plot_verb = True
save_verb = True

print(f"Velocity Autocorrelation Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_autocorrelation.ipynb

# RADIAL DISTRIBUTON FUNCTION

In [ ]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

print("Radial Distribution Function Analysis: run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/radial_distribution_function.ipynb

# 3 BODY STRUCTURE METRIC

In [ ]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

%run ./analysis_modules/3body_analysis.ipynb